In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [ ]:
torch.manual_seed(10)
torch.cuda.manual_seed(10)

In [ ]:
#CIFAR10데이터를 train_dataset과 test_dataset으로 torchvision을 통해 불러오시오

train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                          train=False,
                                          transform=transforms.ToTensor())

100%|██████████| 170498071/170498071 [00:03<00:00, 54546419.46it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:

#batch크기를 128로 해서 train_loader을 만드시오 (DataLoader이용)
batchsize = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batchsize)

In [ ]:
# model을 구성하시오 (class로 해도 좋고 바로 sequential을 이용해도 좋습니다.)
#우리의 목표는 cifar10 데이터의 class분류임을 고려해서 모델을 만드시오
#flatten을 이용해서 1차원으로 변환하고 linear 3번이상 사용하고 활성화 함수는 ReLU를 사용하시오
#hidden layer차원은 128으로 하시오(hidden 차원은 사실 마음대로 하셔도 좋습니다)

In [ ]:
model = nn.Sequential(nn.Flatten(),
                      nn.Linear(3*32*32,128),
                      nn.ReLU(),
                      nn.Linear(128, 128),
                      nn.ReLU(),
                      nn.Linear(128, 128),
                      nn.ReLU(),
                      nn.Linear(128, 10),
                      nn.Softmax())

In [ ]:
model = model.to(device)

In [ ]:

#learning_rate를 0.005로 해서 Adam optimizer를 정의하시오
#손실함수는 cross_entropy를 사용하시오
learning_rate = 0.005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
cross_entropy = nn.CrossEntropyLoss()

In [ ]:

#epoch는 5로해서 학습진행하고 각 epoch마다 accuracy를 출력하시오
num_B = 5
loss_vec = np.zeros(num_B)
optimizer.zero_grad()

for i in range(num_B):
  val_vec = []
  for j, (data, target) in enumerate(train_loader):
    data=data.to(device)
    target=target.to(device)
    output = model(data)
    loss = cross_entropy(output, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    predicted = torch.max(output.data,1)
    v =((predicted.indices == target).sum()).item()/len(target)
    val_vec.append(v)
  print ("Accuracy: {:.4f}".format(np.array(val_vec).mean()))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Accuracy: 0.2312
Accuracy: 0.2879
Accuracy: 0.2960
Accuracy: 0.3010
Accuracy: 0.3100


In [ ]:

#앞에서 받은 test_dataset을 이용해서 test_loader을 만들고 testset의 (평균)정확도를 출력하시오
test_loader = torch.utils.data.DataLoader(test_dataset,batchsize)

In [ ]:

val_vec=[]
for i, (data,target) in enumerate(test_loader):
    data=data.to(device)
    target=target.to(device)
    output=model(data)
    predicted = torch.max(output.data,1)
    v = ((predicted.indices == target).sum()).item()/len(target)
    val_vec.append(v)
print ("Accuracy: {:.4f}".format(np.array(val_vec).mean()))

Accuracy: 0.2738
